# SigLIP Baseline

In [1]:
import dataset as ds
from PIL import Image
import requests
from transformers import AutoProcessor, AutoModel
import torch
from torch.utils.data import DataLoader
from torchvision import transforms as tt
import numpy as np

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [28]:
model = AutoModel.from_pretrained("google/siglip-base-patch16-224").to(device).eval()
processor = AutoProcessor.from_pretrained("google/siglip-base-patch16-224")

In [35]:
config = model.config.to_dict()

print('** LOAD MODEL ** ')
print(f'\tDevice: {device} ({torch.cuda.device_count()} gpus)')
print(f"\tModel parameters: {np.sum([int(np.prod(p.shape)) for p in model.parameters()]):,}")
print(f"\tInput resolution: {config['vision_config']['image_size']}")
print(f"\tContext length: {config['text_config']['max_position_embeddings']}")
print(f"\tVocab size: {config['text_config']['vocab_size']}")

** LOAD MODEL ** 
	Device: cuda (1 gpus)
	Model parameters: 203,155,970
	Input resolution: 224
	Context length: 64
	Vocab size: 32000


In [36]:
scale = tt.Resize((224, 224))
tensor = tt.ToTensor()
image_composed = tt.transforms.Compose([scale, tensor])

test_set = ds.VisualWSDDataset(mode="test", image_transform=image_composed)
test_loader = DataLoader(test_set, batch_size=1, shuffle=False)

In [37]:
# if k is 1 gives all instances with the correct prediction as top prediction
# if k > 1 the correct prediction is in the top k predictions of the model
def hit(results, k):
    counter = 0

    for r in results:
        sims = np.absolute(r[1])
        sorted = np.argsort(sims)[:k]

        if r[0] in sorted:
            counter += 1

    return counter / len(results)

def mrr(results):
    sum = 0

    for r in results:
        sims = np.absolute(r[1])
        sorted = np.argsort(sims)
        sum += 1/(np.where(sorted==r[0])[0][0]+1)

    return sum / len(results)

In [55]:
# if k is 1 gives all instances with the correct prediction as top prediction
# if k > 1 the correct prediction is in the top k predictions of the model
def hit_max(results, k):
    counter = 0

    for r in results:
        sims = np.absolute(r[1])
        sorted = np.argsort(sims)[::-1][:k]

        if r[0] in sorted:
            counter += 1

    return counter / len(results)

def mrr_max(results):
    sum = 0

    for r in results:
        sims = np.absolute(r[1])
        sorted = np.argsort(sims)[::-1]
        sum += 1/(np.where(sorted==r[0])[0][0]+1)

    return sum / len(results)

In [48]:
cos_distance = torch.nn.CosineSimilarity(dim=1, eps=1e-6)
a = torch.randn(1, 10)
cos_distance(a, a)

tensor([1.0000])

In [51]:
def test():
    #correct = 0
    #total = 0
    # since we're not training, we don't need to calculate the gradients for our outputs
    with torch.no_grad():

        results = []
        correct = 0
        max_correct = 0

        for a,data in enumerate(test_loader):
            images = data["imgs"]
            text = data["label_context"][0]
            correct_idx = data["correct_idx"].item()


            print("----------------------------")
            print("batch: " + str(a+1) + "/" + str(len(test_loader)))
            print("label: " + str(text))
            print("correct index: " + str(correct_idx))

            input_text = processor(text=text, padding="max_length", return_tensors="pt")

            input_imgs = []
            for img in images:
                input_imgs.append(processor(images=img, padding="max_length", return_tensors="pt"))

            img_output = []
            for img in input_imgs:
                img_output.append(model.get_image_features(**img.to(device)))

            text_output = model.get_text_features(**input_text.to(device))

            cos_distance = torch.nn.CosineSimilarity(dim=1, eps=1e-6)

            print("text output shape: " + str(text_output.shape))
            print("img output shape: " + str(img_output[0].shape))
            #print(text_output)
            #print(img_output[0])
            
            sims = []
            for i in range(len(img_output)):
                sims.append(cos_distance(text_output, img_output[i]).cpu().item())
                #print(sims[-1]*100)

            sims = np.absolute(np.array(sims))
            min_idx = np.argmin(sims)
            max_idx = np.argmax(sims)

            print("sims: " + str(sims))
            print("min val: " + str(sims[min_idx]) + " min index: " + str(min_idx))
            print("max val: " + str(sims[max_idx]) + " max index: " + str(max_idx))

            results.append((correct_idx, sims))

            if min_idx == correct_idx:
                correct += 1

            if max_idx == correct_idx:
                max_correct += 1

            if a > 49:
                break

    print(correct)
    print(max_correct)

    return results

In [52]:
results = test()

----------------------------
batch: 1/463
label: football goal
correct index: 8
text output shape: torch.Size([1, 768])
img output shape: torch.Size([1, 768])
sims: [0.01201214 0.00696102 0.00855939 0.00261793 0.01111051 0.0041565
 0.01049833 0.00833641 0.00351655 0.00301043]
min val: 0.0026179300621151924 min index: 3
max val: 0.01201214361935854 max index: 0
----------------------------
batch: 2/463
label: mustard seed
correct index: 0
text output shape: torch.Size([1, 768])
img output shape: torch.Size([1, 768])
sims: [0.000695   0.00301303 0.00447044 0.00840998 0.00415324 0.00294004
 0.00199639 0.00711108 0.00408422 0.00464698]
min val: 0.0006949994713068008 min index: 0
max val: 0.008409984409809113 max index: 3
----------------------------
batch: 3/463
label: eating seat
correct index: 5
text output shape: torch.Size([1, 768])
img output shape: torch.Size([1, 768])
sims: [0.03638592 0.0299382  0.03149615 0.02953644 0.03757387 0.03442683
 0.03159223 0.03254194 0.03308138 0.0323025

In [54]:
print("hit1: " + str(hit(results, 1)))
print("mrr: " + str(mrr(results)))
# mit absolutbetrag
# mit richtiger Bildgröße

hit1: 0.13725490196078433
mrr: 0.3214363523187052


In [56]:
print("hit1: " + str(hit_max(results, 1)))
print("mrr: " + str(mrr_max(results)))

hit1: 0.0784313725490196
mrr: 0.28479614067849357


In [25]:
# calcualte MRR
def mrr(results):

    rr = 0

    for r in results:
        correct_idx = r[0]
        sims = r[1]
        sim_correct = sims[correct_idx]
        sim_sort = np.sort(sims)
        new_idx = np.where(sim_sort == sim_correct)
        rr += 1/(new_idx+1)

    return 1/len(results)*mrr

# calculate HIT@1
def hit1(results):
    correct = 0
    for r in results:
        correct_idx = r[0]
        sims = r[1]
        if correct_idx == np.argmin(sims):
            correct += 1

    return correct/len(results)

print(hit1(results))
print(mrr(results))

0.11663066954643629


TypeError: can only concatenate tuple (not "int") to tuple

In [ ]:




url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)

texts = ["a photo of 2 cats", "a photo of 2 dogs"]
# important: we pass `padding=max_length` since the model was trained with this
inputs = processor(text=texts, images=image, padding="max_length", return_tensors="pt")

with torch.no_grad():
    outputs = model(**inputs)

logits_per_image = outputs.logits_per_image
probs = torch.sigmoid(logits_per_image) # these are the probabilities
print(f"{probs[0][0]:.1%} that image 0 is '{texts[0]}'")

31.9% that image 0 is 'a photo of 2 cats'
